## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-16-22-11-19 +0000,nypost,BB gun-wielding man fatally shot by off-duty c...,https://nypost.com/2025/08/16/us-news/bb-gun-w...
1,2025-08-16-22-07-32 +0000,startribune,Eastbound I-94 open between I-35W in Minneapol...,https://www.startribune.com/eastbound-i-94-ope...
2,2025-08-16-22-07-14 +0000,bbc,Beyond chow mein: How a new wave of restaurant...,https://www.bbc.com/news/articles/cy4dk1z02w7o...
3,2025-08-16-22-02-28 +0000,nyt,Democrats Gather for Protests Over Trump’s Pus...,https://www.nytimes.com/2025/08/16/us/politics...
4,2025-08-16-21-58-57 +0000,nypost,"Iran, China are increasing military cooperatio...",https://nypost.com/2025/08/16/world-news/iran-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2345/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
36,trump,61
61,putin,40
57,ukraine,24
73,strike,12
72,canada,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
48,2025-08-16-18-48-22 +0000,nypost,How Trump’s meeting with Putin pivoted from Uk...,https://nypost.com/2025/08/16/us-news/inside-h...,170
164,2025-08-16-01-50-10 +0000,bbc,"No ceasefire, no deal: What summit means for T...",https://www.bbc.com/news/articles/clyvd3gkg1po...,170
176,2025-08-16-00-14-50 +0000,nypost,"Putin tries to woo Trump at Alaska meeting, cl...",https://nypost.com/2025/08/15/us-news/putin-tr...,169
134,2025-08-16-08-01-18 +0000,nyt,Trump and Putin had an isolated conversation i...,https://www.nytimes.com/live/2025/08/15/world/...,167
149,2025-08-16-03-36-11 +0000,nyt,"At Trump’s Summit, No Deal on Ukraine, and No ...",https://www.nytimes.com/2025/08/15/us/politics...,163


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
48,170,2025-08-16-18-48-22 +0000,nypost,How Trump’s meeting with Putin pivoted from Uk...,https://nypost.com/2025/08/16/us-news/inside-h...
25,61,2025-08-16-20-15-27 +0000,wapo,Canada orders Air Canada flight attendants bac...,https://www.washingtonpost.com/world/2025/08/1...
34,58,2025-08-16-19-41-53 +0000,nypost,Trump’s efforts at Alaska summit applauded by ...,https://nypost.com/2025/08/16/world-news/trump...
66,31,2025-08-16-16-37-41 +0000,nypost,Zelensky warns Putin will step up attacks on U...,https://nypost.com/2025/08/16/world-news/zelen...
61,31,2025-08-16-17-06-24 +0000,nypost,NYPD drone allows Mayor Adams to see real-time...,https://nypost.com/2025/08/16/us-news/drone-al...
0,28,2025-08-16-22-11-19 +0000,nypost,BB gun-wielding man fatally shot by off-duty c...,https://nypost.com/2025/08/16/us-news/bb-gun-w...
6,28,2025-08-16-21-40-53 +0000,wapo,"Trump drops ceasefire demand for Ukraine war, ...",https://www.washingtonpost.com/world/2025/08/1...
39,27,2025-08-16-19-28-00 +0000,wsj,More National Guard troops are heading soon to...,https://www.wsj.com/politics/national-security...
58,23,2025-08-16-18-00-00 +0000,nypost,Greedy landlords — not Bronx residents — were ...,https://nypost.com/2025/08/16/us-news/greedy-l...
185,23,2025-08-15-23-06-54 +0000,bbc,'Bus fares have cut my food budget': Under-22s...,https://www.bbc.com/news/articles/c70xjj8746wo...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
